In [1]:
import pandas as pd
import numpy as np
from io import StringIO

Files imported: Students_export.csv, Student_race.csv <br>
File exported: eIEPPROStudentDemographics.csv

<H4> Things to do: <br>
* Validate students + RED & Hispanic Y/N data (largely done) <br>
* encapsulate all text in quotes (needs to be done) <br>
* Add student demographic info & Parent/Guardian contacts afterwards

<H5> StudentRace Table: converted to binary values 
    
| STUDENTRACE.StudentID |STUDENTRACE.RaceCd | ---|
|--- |--- |--- |
| Some# | Possibilities are: |
| AS | Asian | 
| WH | White | 
| BL | Black | 
| AM | American Native | 
| PI | Pacific Islander | 
| not in Student Race table | Hispanic |

Appendix 1 Spec (06/2022 Document v22)

| eIEPPRO Data Field | Powerschool Table Name | Details | 
| --- |--- |--- |
| StateIDNumber | STUDENTS.State_StudentNumber | nvarchar (10) |
| SISIDNumber   | STUDENTS.Student_Number      | nvarchar(9)   |
| SchoolCode    | STUDENTS.SchoolID            | nvarchar(9)   |  
| LastName      | STUDENTS.Last_Name           | nvarchar(40)  |
| FirstName     | STUDENTS.First_Name          | nvarchar(25)  |
| MiddleName    | STUDENTS.Middle_Name         | nvarchar(25)  |
| DOB           | STUDENTS.DOB                 | smalldatetime |
| Gender        | STUDENTS.Gender              | nvarchar(6)*  |  
| Ethnicity_HI  | STUDENTS.Hispanic Y/N        | Yes: HI == 1 |
| Ethnicity_AM  | STUDENTRACE.RaceCd == AM |--- |
| Ethnicity_AS  | STUDENTRACE.RaceCd == AS |--- |
| Ethnicity_BL  | STUDENTRACE.RaceCd == BL |--- |
| Ethnicity_PI  | STUDENTRACE.RaceCd == PI |--- |
| Ethnicity_WH  | STUDENTRACE.RaceCd == AM |--- |
| Grade | STUDENTS.Grade_Level |--- |
| Student Address #1          | STUDENTS.Mailing_Street     | Powerschool  ~ Street Address |
| Student Address #2          | Students.Mailing_City, Mailing_State and Mailing_Zip     |  |
| Student Physical Address ~ StudentPhyAddress  | STUDENTS.Street   | STUDENTS.Street |  
| Student Phone #             | StudentPhone        | STUDENTS.Home_Phone |
| Student E-Mail              | StudentEmail        | will not provide |
| Parent #1 Name              | Parent1Name         | --- |
| Parent #1 Address-1         | Parent1Address1     | Street Address |
| Parent #1 Address-2         | Parent1Address2     | City, State and Zip Code |  
| Parent #1 Home Phone        | Parent1HomePhone    | --- |
| Parent #1 Work Phone        | Parent1WorkPhone    | --- |
| Parent #1 Cell Phone        | Parent1CellPhone    | --- |
| Parent #1 Relationship      | Parent1Relationship | --- |
| Parent #1 Email             | Parent1Email        | --- |
| Parent #2 Name              | Parent2Name         | --- |
| Parent #2 Address-1         | Parent2Address1     | Street Address |
| Parent #2 Address-2         | Parent2Address2     | City, State and Zip Code |  
| Parent #2 Home Phone        | Parent2HomePhone    | --- |
| Parent #2 Work Phone        | Parent2WorkPhone    | --- |
| Parent #2 Cell Phone        | Parent2CellPhone    | --- |
| Parent #2 Relationship      | Parent2Relationship | --- |
| Parent #2 Email             | Parent2Email        | --- |
| PrimaryLanguage | S_AZ_STU_X.spokenlanguage |--- |
| HomeLanguage | S_AZ_STU_X.homelanguagecode |--- |
| District of Residence | DOR |--- |
| District of Attendance | DOA |--- |
| Home School | HomeSchool |--- |
| Attending School | AttendingSchool |--- |
| Emergency Contact | EmergencyContact |--- |
| Emergency Contact Phone | EmergencyContactPhone |--- |

* Gender: AzEDS defined Gender values <br>
* Students pulled based on ACTIVITIES.childfind504 == 1 or ACTIVITIES.childfindidea == 1 <br>
S_AZ_STU_X.firstacquiredlanguage <br>
S_AZ_STU_X.homelanguagecode is home language <br>
S_AZ_STU_X.spokenlanguage is the same as primary language <br>


<H3> Input Files: <br>

In [2]:
Students = pd.read_csv("Students_export.csv", encoding='latin-1')
StudentRaceAndEthnicity = pd.read_csv("StudentRace.csv", encoding='latin-1')

<H5> All students with the flags for childfind504 and childfindidea are pulled in. These flags are to be kept on for the time that students have a 504 or IEP. <br>
    
<H4> Consider running tests to compare Flag population to Population at Large - compare numbers

In [3]:
Students_504 = Students.loc[Students['ACTIVITIES.childfind504'] == 1] 
Students_and_idea = Students.loc[Students['ACTIVITIES.childfindidea'] == 1]

In [4]:
# Combine communities and drop duplicates
df_504_id = pd.concat([Students_504, Students_and_idea], ignore_index=True).drop_duplicates()

In [5]:
# Clean columns
df_504_id.columns = df_504_id.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('/', '')

In [6]:
# Create the dataframe for writing, column by column
df_StudentData = df_504_id.filter(['students.state_studentnumber', 
                                  'students.student_number',
                                  'students.id',
                                  'students.schoolid',
                                  'students.last_name',
                                  'students.first_name',
                                  'students.middle_name',
                                  'students.dob',
                                  'students.gender',
                                  'hispanic_yn', #insert red data later                   
                                  'students.grade_level',
                                  'students.mailing_street',
                                  'students.mailing_city',
                                  'students.mailing_state',
                                  'students.mailing_zip',
                                  'students.street', # Using for physical street
                                  'students.home_phone', #Considering inserting contrived email
                                  's_az_stu_x.homelanguagecode',
                                  's_az_stu_x.spokenlanguage',
                                  'ell_alert',
                                  '504_alert'
                                 ])

In [7]:
df_StudentData.set_index('students.student_number')

,students.state_studentnumber,students.id,students.schoolid,students.last_name,students.first_name,students.middle_name,students.dob,students.gender,hispanic_yn,students.grade_level,students.mailing_street,students.mailing_city,students.mailing_state,students.mailing_zip,students.street,students.home_phone,s_az_stu_x.homelanguagecode,s_az_stu_x.spokenlanguage,ell_alert,504_alert
students.student_number,,,,,,,,,,,,,,,,,,,,
90001285,43450183,55,450,Jensen,Cash,Wyatt,2008-12-17,M,0,8,3317 S Higley Rd #114-609,Gilbert,AZ,85297,3892 E Cherry Hill Dr,480-233-2754,0,NaN,NaN,Y
20081142,39785428,1500,100,Napier,Timothy,Arthur,2009-03-08,M,0,8,20206 N Pepka Dr.,Maricopa,AZ,85138,20206 N Pepka Dr,152-041-4839,0,NaN,NaN,Y
70001095,43198443,2621,350,Dohaniuk,Jordan,Jacob,2009-06-01,M,0,8,5613 W Copperhead Dr.,Tucson,AZ,85742,5613 W Copperhead Dr,520-603-9047,0,NaN,NaN,Y
70001183,43199075,2623,350,Roberts,Katelyn,Amy,2009-01-09,F,0,8,10255 N Lambert Ct.,Tucson,AZ,85742,10255 N Lambert Ct,480-529-7959,0,NaN,NaN,Y
70001083,39663726,2655,350,Green,Cody,Jerry Alan,2008-12-18,M,0,8,515 E Parker Point Pl,Oro Valley,AZ,85737,515 E Parker Point Pl,520-331-6511,0,NaN,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76386,91968642,224977,500,Farooqi,Christopher,Amir,2010-11-22,M,0,6,16716 N 113th Dr,Surprise,AZ,85378,16716 N 113th Dr,602-592-5478,0,0.0,NaN,NaN
76392,47748746,224983,1150,Davy,Adalynn,Agatha-Elwell,2012-12-29,F,0,4,1927 W Fruit Tree Ln,San Tan Valley,AZ,85142,1927 W Fruit Tree Ln,480-216-1744,0,0.0,NaN,NaN
76396,42881544,224987,1150,White,Jonathan,Louis,2009-03-06,M,0,7,19919 E Stacey Rd,Queen Creek,AZ,85142,19919 E Stacey Rd,602-573-0547,0,0.0,NaN,NaN


In [8]:
df_StudentData

,students.state_studentnumber,students.student_number,students.id,students.schoolid,students.last_name,students.first_name,students.middle_name,students.dob,students.gender,hispanic_yn,...,students.mailing_street,students.mailing_city,students.mailing_state,students.mailing_zip,students.street,students.home_phone,s_az_stu_x.homelanguagecode,s_az_stu_x.spokenlanguage,ell_alert,504_alert
0,43450183,90001285,55,450,Jensen,Cash,Wyatt,2008-12-17,M,0,...,3317 S Higley Rd #114-609,Gilbert,AZ,85297,3892 E Cherry Hill Dr,480-233-2754,0,NaN,NaN,Y
1,39785428,20081142,1500,100,Napier,Timothy,Arthur,2009-03-08,M,0,...,20206 N Pepka Dr.,Maricopa,AZ,85138,20206 N Pepka Dr,152-041-4839,0,NaN,NaN,Y
2,43198443,70001095,2621,350,Dohaniuk,Jordan,Jacob,2009-06-01,M,0,...,5613 W Copperhead Dr.,Tucson,AZ,85742,5613 W Copperhead Dr,520-603-9047,0,NaN,NaN,Y
3,43199075,70001183,2623,350,Roberts,Katelyn,Amy,2009-01-09,F,0,...,10255 N Lambert Ct.,Tucson,AZ,85742,10255 N Lambert Ct,480-529-7959,0,NaN,NaN,Y
4,39663726,70001083,2655,350,Green,Cody,Jerry Alan,2008-12-18,M,0,...,515 E Parker Point Pl,Oro Valley,AZ,85737,515 E Parker Point Pl,520-331-6511,0,NaN,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,91968642,76386,224977,500,Farooqi,Christopher,Amir,2010-11-22,M,0,...,16716 N 113th Dr,Surprise,AZ,85378,16716 N 113th Dr,602-592-5478,0,0.0,NaN,NaN
2624,47748746,76392,224983,1150,Davy,Adalynn,Agatha-Elwell,2012-12-29,F,0,...,1927 W Fruit Tree Ln,San Tan Valley,AZ,85142,1927 W Fruit Tree Ln,480-216-1744,0,0.0,NaN,NaN
2625,42881544,76396,224987,1150,White,Jonathan,Louis,2009-03-06,M,0,...,19919 E Stacey Rd,Queen Creek,AZ,85142,19919 E Stacey Rd,602-573-0547,0,0.0,NaN,NaN
2626,22178438,76408,225011,1000,Hudson,Deshawn,James,2012-11-09,M,0,...,"2902 W Sweetwater Ave, Apt 2031",Phoenix,AZ,85029,"2902 W Sweetwater Ave, Apt 2031",470-967-9423,0,0.0,NaN,NaN


In [9]:
df_StudentData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2616 entries, 0 to 2627
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   students.state_studentnumber  2616 non-null   int64  
 1   students.student_number       2616 non-null   int64  
 2   students.id                   2616 non-null   int64  
 3   students.schoolid             2616 non-null   int64  
 4   students.last_name            2616 non-null   object 
 5   students.first_name           2616 non-null   object 
 6   students.middle_name          2368 non-null   object 
 7   students.dob                  2616 non-null   object 
 8   students.gender               2616 non-null   object 
 9   hispanic_yn                   2616 non-null   int64  
 10  students.grade_level          2616 non-null   int64  
 11  students.mailing_street       2616 non-null   object 
 12  students.mailing_city         2616 non-null   object 
 13  stu

<H5> Transforming Columns

In [10]:
# Clean data as appropriate
# Turn State_StudentNumber to varchar
# df["Fee"]=df["Fee"].values.astype('string')
df_StudentData["stu_id"]        = df_StudentData["students.id"].values.astype('int')
df_StudentData["StateIDNumber"] = df_StudentData["students.state_studentnumber"].values.astype('int')
df_StudentData["SISIDNumber"]   = df_StudentData["students.student_number"].values.astype('int')
df_StudentData["SchoolCode"]    = df_StudentData["students.schoolid"].values.astype('int')
df_StudentData["LastName"]      = df_StudentData["students.last_name"]
df_StudentData["FirstName"]     = df_StudentData["students.first_name"]
df_StudentData["MiddleName"]    = df_StudentData["students.middle_name"]
df_StudentData["DOB"]           = pd.to_datetime(df_StudentData["students.dob"])
df_StudentData["Gender"]        = df_StudentData["students.gender"]
df_StudentData["Ethnicity_HI"]  = df_StudentData["hispanic_yn"]
# Break to insert RED Data

df_StudentData["Grade"]           = df_StudentData["students.grade_level"].values.astype('int')
df_StudentData["StudentAddress1"] = df_StudentData["students.mailing_street"]
df_StudentData["StudentAddress2"] = df_StudentData['students.mailing_city'].astype(str) + ', ' + df_StudentData['students.mailing_state'] + ', ' + df_StudentData['students.mailing_zip']
df_StudentData["StudentPhyAddress"] = df_StudentData["students.street"]
df_StudentData["StudentPhone"]      = df_StudentData["students.home_phone"]
df_StudentData["StudentEMail"]      = ""
# Insert Parent Contact Data

df_StudentData["PrimaryLanguage"]  = df_StudentData["s_az_stu_x.spokenlanguage"] #.values.astype('str') # clearing out int to see what happens
df_StudentData["HomeLanguage"]     = df_StudentData["s_az_stu_x.homelanguagecode"] # .values.astype('int')



<H5> Primary and Home Language Transformations:

In [11]:
transform_language = {
    0:   'English', 
    1:   'Spanish', 
    2:   'Arabic', 
    3:   'Cambodian', 
    4:   'Cantonese', 
    5:   'Czechoslovakian', 
    6:   'Dutch', 
    7:   'Filipino', 
    8:   'Finnish', 
    9:   'French', 
    10:   'German', 
    11:   'Greek', 
    12:   'Hebrew', 
    13:   'Hungarian', 
    14:   'Italian', 
    15:   'Japanese', 
    16:   'Korean', 
    17:   'Laotian', 
    18:   'Mandarin', 
    19:   'Polish', 
    20:   'Portuguese', 
    21:   'Romanian', 
    22:   'Russian', 
    23:   'Thai', 
    24:   'Ukrainian', 
    25:   'Vietnamese', 
    26:   'Serbo-Croatian', 
    27:   'Other Non-Indian', 
    28:   'Navajo', 
    29:   'Apache (San Carlos)', 
    30:   'Apache (Whiteriver)', 
    31:   'Chemehuevi', 
    32:   'Cocopah', 
    33:   'Hopi', 
    34:   'Haulapai', 
    35:   'Kaibab-Paiute', 
    36:   'Maricopa', 
    37:   'Mohave', 
    38:   'Papago (Tohono Oodham)',
    39:   'Pima (Akimel Oodhman)',
    40:   'Supai', 
    41:   'Tewa', 
    42:   'Yaqui', 
    43:   'Yavapai', 
    44:   'Yuma', 
    45:   'Other Indian', 
    46:   'American Sign Language',
    47:   'Albanian', 
    48:   'Amharic', 
    49:   'Azeri', 
    50:   'Burmese', 
    51:   'Dari', 
    52:   'Dinka', 
    53:   'Farsi', 
    54:   'Kirundi', 
    55:   'Kru', 
    56:   'Lingala', 
    57:   'Maay', 
    58:   'Mandingo', 
    59:   'Nuer', 
    60:   'Pashto', 
    61:   'Serbian', 
    62:   'Somali', 
    63:   'Sawhili', 
    64:   'Tagalog', 
    65:   'Urdu', 
    66:   'Uzbek', 
    67:   'Vai', 
    68:   'Afrikaans', 
    69:   'Bengali', 
    70:   'Hindi', 
    71:   'Persian', 
    72:   'Punjabi', 
    73:   'Cherokee', 
    74:   'Armenian', 
    75:   'Bosnian', 
    76:   'Central Khmer', 
    77:   'Chuukese', 
    78:   'Gujarati', 
    79:   'Haitian; Haitian Creole', 
    80:   'Hmong; Mong', 
    81:   'Karen Languages', 
    82:   'Lao', 
    83:   'Malayalam', 
    84:   'Marshallese', 
    85:   'Nepali', 
    86:   'Otomian languages', 
    87:   'Samoan', 
    88:   'Tamil', 
    89:   'Telugu', 
    90:   'Tigrinya', 
    91:   'Turkish', 
    92:   'undeclared', 
    93:   'Twi', 
    94:   'Yupik languages'
}

df_StudentData["PrimaryLanguage"] = df_StudentData["PrimaryLanguage"].map(transform_language)
df_StudentData["HomeLanguage"] = df_StudentData["HomeLanguage"].map(transform_language)

<H5> District Transformations: <br>
District of Residence & Attendance: based on School ID, but LOA gets Surprise (double check)

In [12]:
transform_schoolID = {
    100: 'Legacy Traditional School - Maricopa',
    150: 'Legacy Traditional School - Casa Grande',
    200: 'Legacy Traditional School - Queen Creek',
    250: 'Legacy Traditional School - Avondale',
    300: 'Legacy Traditional School - Chandler',
    350: 'Legacy Traditional School - NW Tucson',
    400: 'Legacy Traditional School - Laveen',
    450: 'Legacy Traditional School - Gilbert',
    500: 'Legacy Traditional School - Surprise',
    550: 'Legacy Traditional School - North Chandler',
    600: 'Legacy Traditional School - Glendale',
    650: 'Legacy Traditional School - Peoria',
    700: 'Legacy Traditional School - East Mesa',
    750: 'Legacy Traditional School - Phoenix',
    800: 'Legacy Traditional School - Goodyear',
    850: 'Legacy Traditional School - North Phoenix',
    900: 'Legacy Traditional School - West Surprise',
    950: 'Legacy Traditional School - East Tucson',
    1000: 'Legacy Traditional School - Deer Valley',
    1050: 'Legacy Traditional School - Mesa',
    1100: 'Legacy Online Academy', 
    1150: 'Legacy Traditional School - San Tan'
}

# This where LOA should get Surprise data
df_StudentData['DOR'] = df_StudentData['SchoolCode'].map(transform_schoolID)
df_StudentData['DOA'] = df_StudentData['SchoolCode'].map(transform_schoolID)

# Home School and Attending School his is where LOA stays LOA - plus students that are exceptions to the rule?
df_StudentData['HomeSchool'] = df_StudentData['SchoolCode'].map(transform_schoolID)
df_StudentData['AttendingSchool'] = df_StudentData['SchoolCode'].map(transform_schoolID)


In [13]:
df_StudentData['EmergencyContact'] = ''
df_StudentData['EmergencyContactPhone'] = ''

<H4> Last Fields from Schools_Export - these students should have ChildFing Flags preserved: <br>
    * 504Student (comes from '504 Alert'), comes in as Y, translate to 1 <br> 
    * EL Student (comes from 'ELL Alert') <br>

In [14]:
# sampleDF.housing.replace(('yes', 'no'), (1, 0), inplace=True)
df_StudentData["504Student"]       = df_StudentData['504_alert']
df_StudentData['504Student'].replace(('Y', ''), (1, 0), inplace=True)

In [15]:
df_StudentData["ELStudent"]        = df_StudentData['ell_alert']

<H3> Student Race and Ethnicity data <br>
<H5> HI, AM, AS, BL, PI, WH must be converted to 1 = YES or 0 = No values, per student <br>
    based on STUDENTRACE.StudentID <br>
    STUDENTS.ID == STUDENTRACE.StudentID <br>

In [16]:
# Create dataframe - this version works!
def BuildRaceMatrix8( df_StudentRace ):
    global df_RED
    df_RED = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_StudentRace.iterrows():
        student_ID = row['STUDENTRACE.StudentID']
        race_data = row['STUDENTRACE.RaceCd']
        df_RED.at[index, 'stu_id'] = student_ID
        
        match race_data:
            case 'AM':
                df_RED.at[index, 'AM'] = 1
            case 'AS':
                df_RED.at[index, 'AS'] = 1
            case 'BL':
                df_RED.at[index, 'BL'] = 1
            case 'PI':
                df_RED.at[index, 'PI'] = 1
            case 'WH':
                df_RED.at[index, 'WH'] = 1
    return df_RED

In [17]:
testRed3 = BuildRaceMatrix8(StudentRaceAndEthnicity).fillna(0).astype(int)

In [18]:
df_pivot = pd.pivot_table(testRed3, values=['stu_id', 'AS', 'WH', 'BL', 'AM', 'PI'], index=['stu_id'], aggfunc=np.sum, margins=True)

In [22]:
df_Pivot_plus_StudentData = df_StudentData.join(df_pivot, on='stu_id')

In [21]:
df_towrite.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2616 entries, 0 to 2627
Data columns (total 52 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   students.state_studentnumber  2616 non-null   int64         
 1   students.student_number       2616 non-null   int64         
 2   students.id                   2616 non-null   int64         
 3   students.schoolid             2616 non-null   int64         
 4   students.last_name            2616 non-null   object        
 5   students.first_name           2616 non-null   object        
 6   students.middle_name          2368 non-null   object        
 7   students.dob                  2616 non-null   object        
 8   students.gender               2616 non-null   object        
 9   hispanic_yn                   2616 non-null   int64         
 10  students.grade_level          2616 non-null   int64         
 11  students.mailing_street       

<H1> Lastly, Encapsulate Strings with quotes, Fill in NaN values with 0

In [20]:
df_towrite.to_csv('IEPPRO_Draft_forUpload.csv', encoding='utf-8')